In [1]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [3]:
!kaggle datasets download -d jcprogjava/handwritten-digits-dataset-not-in-mnist


Dataset URL: https://www.kaggle.com/datasets/jcprogjava/handwritten-digits-dataset-not-in-mnist
License(s): CC-BY-SA-4.0
  0% 0.00/54.6M [00:00<?, ?B/s]
100% 54.6M/54.6M [00:00<00:00, 1.76GB/s]


In [4]:
!unzip -q handwritten-digits-dataset-not-in-mnist.zip -d data
!ls data


dataset


In [5]:
import tensorflow as tf
import os

DATA_DIR = "data/dataset"
IMG_SIZE = (28, 28)
BATCH = 64
SEED = 42

train_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    labels="inferred",
    label_mode="int",
    color_mode="grayscale",
    image_size=IMG_SIZE,
    batch_size=BATCH,
    validation_split=0.2,
    subset="training",
    seed=SEED,
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    labels="inferred",
    label_mode="int",
    color_mode="grayscale",
    image_size=IMG_SIZE,
    batch_size=BATCH,
    validation_split=0.2,
    subset="validation",
    seed=SEED,
)

def normalize(x, y):
    return tf.cast(x, tf.float32) / 255.0, y

train_ds = train_ds.map(normalize).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.map(normalize).prefetch(tf.data.AUTOTUNE)

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28,28,1)),
    tf.keras.layers.Conv2D(32,3,activation="relu",padding="same"),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64,3,activation="relu",padding="same"),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation="relu"),
    tf.keras.layers.Dense(10,activation="softmax")
])

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.fit(train_ds, validation_data=val_ds, epochs=8)

os.makedirs("model", exist_ok=True)
model.save("model/digit_cnn.keras")

print("✅ Training done, model saved")


Found 107730 files belonging to 10 classes.
Using 86184 files for training.
Found 107730 files belonging to 10 classes.
Using 21546 files for validation.
Epoch 1/8
1347/1347 ━━━━━━━━━━━━━━━━━━━━ 19s 12ms/step - accuracy: 0.1002 - loss: 2.3026 - val_accuracy: 0.0969 - val_loss: 2.3027
Epoch 2/8
1347/1347 ━━━━━━━━━━━━━━━━━━━━ 13s 9ms/step - accuracy: 0.0989 - loss: 2.3027 - val_accuracy: 0.0969 - val_loss: 2.3027
Epoch 3/8
1347/1347 ━━━━━━━━━━━━━━━━━━━━ 20s 9ms/step - accuracy: 0.1005 - loss: 2.3028 - val_accuracy: 0.0969 - val_loss: 2.3027
Epoch 4/8
1347/1347 ━━━━━━━━━━━━━━━━━━━━ 20s 9ms/step - accuracy: 0.0999 - loss: 2.3028 - val_accuracy: 0.0969 - val_loss: 2.3027
Epoch 5/8
1347/1347 ━━━━━━━━━━━━━━━━━━━━ 12s 9ms/step - accuracy: 0.1003 - loss: 2.3028 - val_accuracy: 0.0969 - val_loss: 2.3027
Epoch 6/8
1347/1347 ━━━━━━━━━━━━━━━━━━━━ 13s 9ms/step - accuracy: 0.1006 - loss: 2.3027 - val_accuracy: 0.0969 - val_loss: 2.3027
Epoch 7/8
1347/1347 ━━━━━━━━━━━━━━━━━━━━ 12s 9ms/step - accuracy:

In [6]:
!zip -r digit_app_files.zip model


  adding: model/ (stored 0%)
  adding: model/digit_cnn.keras (deflated 69%)


In [7]:
from google.colab import files
files.download("digit_app_files.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>